<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<style>
    h1 {
        line-height: 200%;
        font-family: 'Comic Sans MS', cursive, sans-serif;
        color: #ffb6c1; /* رنگ نارنجی پاستیلی */
    }
</style>
</head>
<body>
    <h1>Zahra Alinaghipour</h1>
</body>
</html>


Building a machine learning model that can predict the topic category of a document based on text available in a link, such as <i>Title</i>, <i>Description</i>, <i>Full Text Content</i>, etc. For example, if we have a link from a news website with the title "<i>کیهان کلهر جایزه مرد سال موسیقی جهان را دریافت کرد</i>," our model should predict that this article is related to the topic of "موسیقی."

In [36]:
import numpy as np
import pandas as pd

|Column|Description|
|:------:|:---:|
|`category`|Topic (target variable)|
|`description`|Description|
|`text_content`|Text content|
|`title`|Title|
|`h1`|Content of the <code>h1</code> tag of the page|
|`h2`|Content of the <code>h2</code> tag of the page|
|`url`|Link address|
|`domain`|Website domain|
|`id`|Link ID|

In [37]:
train = pd.read_csv('../data/yektanet_train.csv')
train.head()

,category,description,text_content,title,h1,h2,url,domain,id
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",معنی از شوبنده ها | جدول یاب از شوبنده ها 381,معنی از شوبنده ها | جدول یاب,معنی از شوبنده ها,از شوبنده ها در معادل ابجد,jadvalyab.ir/search?q=%D8%A7%D8%B2+%D8%B4%D9%8...,jadvalyab.ir,158
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,عکس بیت‌کوین کش برای پروفایل عکس و والپیپرهای ...,عکس بیت‌کوین کش برای پروفایل,عکس بیت‌کوین کش برای پروفایل,عکس بیت کوین با کیفیت 4K عکس ارزهای دیجیتال عک...,jowhareh.com/photo/%D8%B9%DA%A9%D8%B3-%D8%A8%D...,jowhareh.com,3268
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی,آدرس و تلفن دکتر مهناز عابدینی نظرات و تجربیات...,doctor-yab.ir/Search/14773/%D8%AF%DA%A9%D8%AA%...,doctor-yab.ir,175
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,دانلود تحریم‌گذر Geph برای اندروید خانه/اندروی...,دانلود تحریم‌گذر Geph برای اندروید,دانلود تحریم‌گذر Geph برای اندروید,دانلود نرم افزار Geph,palexe.site/dl/geph-android/,palexe.site,3402
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ,راه‌اندازی تلویزیون همگام‌سازی کنترل اتصال به ...,rokhdadeghtesadi.ir/43874/,rokhdadeghtesadi.ir,3811


In [38]:
test = pd.read_csv('../data/yektanet_test.csv')
test.head()

,description,text_content,title,h1,h2,url,domain,id
0,پیام ناشناس یک ابزار رایگان برای ساخت چالش های...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,NaN,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,abzarek.ir/service-p/msg/824757/,abzarek.ir,3291
1,اقتصادنیوز: سید امیر احمد علیه السلام ملقب به ...,حرم شاهچراغ شیراز در گذر زمان+تصاویر حرم شاهچر...,حرم شاهچراغ شیراز در گذر زمان+تصاویر,حرم شاهچراغ شیراز در گذر زمان+تصاویر,NaN,eghtesadnews.com/%D8%A8%D8%AE%D8%B4-%D8%A7%D8%...,eghtesadnews.com,5053
2,آخرین خبر های بین المللی و آخرین تحولات جهان، ...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد ✖...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد,زلنسکی پیام پوتین را دریافت کرد,NaN,akharinkhabar.ir/world/9332333/%D8%B2%D9%84%D9...,akharinkhabar.ir,564
3,آهنگ گاهی دریدن لازم نیست با صدای امیر تتلو با...,آهنگ گاهی دریدن لازم نیست امیر تتلو آهنگ گاهی ...,آهنگ گاهی دریدن لازم نیست امیر تتلو,آهنگ گاهی دریدن لازم نیست امیر تتلو,گاهی دریدن لازم نیست امیر تتلو,behmusic.com/music/%DA%AF%D8%A7%D9%87%DB%8C-%D...,behmusic.com,3607
4,کیهان با اشاره به حوادث و ناآرامیهای اخیر کشور...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,برچسب‌ها نظر شما آخرین اخبار سیاسی ایران در بح...,khabaronline.ir/news/1688132/%D8%A7%D9%86%D8%A...,khabaronline.ir,2358


In [39]:
train['category'].value_counts()

سلامت                  614
ورزش                   514
حقوق و دولت و سیاست    486
هنر و سرگرمی           410
موسیقی                 314
تکنولوژی و کامپبوتر    287
تجارت و اقتصاد         283
فیلم و سینما           239
خودرو                  237
اجتماعی                209
سفر و گردشگری          182
غذا و نوشیدنی          171
مذهبی                  160
مسکن                   131
خانه و باغبانی         128
مد و زیبایی            118
کتاب و ادبیات           83
تحصیلات                 79
اشتغال                  47
علم و دانش              34
خانواده                 34
حیوانات خانگی           29
Name: category, dtype: int64

In [40]:
X_train = train.drop(columns=['category'])
y_train = train['category']

In [41]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
data_resampled, targets_resampled = ros.fit_resample(X_train, y_train)

In [42]:
targets_resampled.value_counts()

کتاب و ادبیات          614
تجارت و اقتصاد         614
اشتغال                 614
خانواده                614
علم و دانش             614
تحصیلات                614
موسیقی                 614
خانه و باغبانی         614
هنر و سرگرمی           614
سفر و گردشگری          614
فیلم و سینما           614
مذهبی                  614
اجتماعی                614
مد و زیبایی            614
خودرو                  614
مسکن                   614
حقوق و دولت و سیاست    614
غذا و نوشیدنی          614
ورزش                   614
تکنولوژی و کامپبوتر    614
سلامت                  614
حیوانات خانگی          614
Name: category, dtype: int64

In [43]:
from stopwords_guilannlp import stopwords_output
persian_stopwords = stopwords_output("Persian", "nar")

In [44]:
from hazm import Normalizer, word_tokenize
import string

normalizer = Normalizer()

def preprocessor(input):
    punc_removed = input.translate(str.maketrans('', '', string.punctuation))
    normalized = normalizer.normalize(punc_removed)
    tokens = word_tokenize(normalized)
    filtered = []
    for token in tokens:
        token = str(token)
        token = token.lower()
        if not token in persian_stopwords and not token.isdigit():
            filtered.append(token)
    output = ' '.join(filtered)
    return output

In [45]:
def tokenizer(text):
    return word_tokenize(text)

In [46]:
from sklearn.preprocessing import LabelEncoder

X_train = data_resampled['title']
y_train = targets_resampled
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)

In [47]:
X_train

0                             معنی از شوبنده ها | جدول یاب
1                             عکس بیت‌کوین کش برای پروفایل
2        دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...
3                       دانلود تحریم‌گذر Geph برای اندروید
4        ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...
                               ...                        
13503             انتشارات فردوسی در باب همایون | کتاب اول
13504                  دانلود pdf کتاب قصر خوشبختی پرل باک
13505                     رمان خدمتکار اجباری من | ❤️یک نت
13506                    معنی معطل | واژگان مترادف و متضاد
13507                            شعر نقاش شهریار | ❤️یک نت
Name: title, Length: 13508, dtype: object

In [48]:
y_train

array([21,  2, 11, ..., 21, 21, 21])

In [49]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline


svm = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor,
                                         analyzer='word', ngram_range=(1, 2),
                                         min_df=5, lowercase=False)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf-svm', LinearSVC(loss='hinge', penalty='l2', max_iter=5000))])

model = svm.fit(X_train, y_train)

In [50]:
model

Pipeline(steps=[('vect',
                 CountVectorizer(lowercase=False, min_df=5, ngram_range=(1, 2),
                                 preprocessor=<function preprocessor at 0x0000026A27947AF0>,
                                 tokenizer=<function tokenizer at 0x0000026A258C60D0>)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf-svm', LinearSVC(loss='hinge', max_iter=5000))])

In [51]:
from sklearn.metrics import f1_score

predictions = model.predict(X_train)
print('Training F1-score:', f1_score(y_train, predictions, average='weighted'))

Training F1-score: 0.9804656537100928


In [52]:
X_test = test['title']
predictions = model.predict(X_test)

In [53]:
submission = pd.DataFrame({'category': encoder.inverse_transform(predictions)})
submission.head()

,category
0,هنر و سرگرمی
1,حقوق و دولت و سیاست
2,حقوق و دولت و سیاست
3,موسیقی
4,مذهبی


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
(Word Cloud)
</font>
</h2>

<center>
<img src="wordcloud.png">
</center>


In [ ]:
cloud_text = ''
for text in X_train:
    cloud_text += text + ' '

In [ ]:
from wordcloud_fa import WordCloudFa

wc = WordCloudFa(width=1200, height=800, persian_normalize=True, include_numbers=False, max_words=120, background_color='white', min_font_size=10, max_font_size=100)
word_cloud = wc.generate(cloud_text)
image = word_cloud.to_image()
image.show()
image.save('wordcloud.png')